# Departamento médico

## Importação das bibliotecas e base de dados

In [2]:
import os # recursos do sistema operacional
import cv2 # OpenCV Visão computacional para fazer leitura de imagens
import tensorflow as tf # Trabalhar com DL e redes neurais
import numpy as np
from tensorflow.keras import layers, optimizers # layers = camadas das redes neurais # optimizers = algoritmos para ajustes dos pesos
from tensorflow.keras.applications import ResNet50 # Tipo de rede neural convolucional
from tensorflow.keras.layers import Input, Dense, AveragePooling2D, Dropout, Flatten 
from tensorflow.keras.models import Model # Criar rede neural
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Acessar diretório de imagens para enviarmos a rede neural
from tensorflow.keras.callbacks import ModelCheckpoint # Salvar os pesos da rede neural
import matplotlib.pyplot as plt
import seaborn as sns

## Visualização dos dados

## Carregamento da rede neural com pesos pré-treinados

## Construção e treinamento do modelo de deep learning

## Avaliação da rede neural

## Classificação de somente uma imagem